In [6]:
import gensim
import numpy as np
import pickle

DATA_PATH = '../database/'

In [7]:
def gensim_load_vec(path="SBW-vectors-300-min5.bin"):
    gensim_emb = gensim.models.KeyedVectors.load_word2vec_format(path, binary=True)
    vocab      = gensim_emb.index2word
    vec        = gensim_emb.syn0
    shape      = gensim_emb.syn0.shape
    return gensim_emb, vec, shape, vocab

In [8]:
#gensim_emb, vec, shape, vocab = gensim_load_vec(DATA_PATH + "embeddings/SBW-vectors-300-min5.bin")

In [7]:
TEMP_PATH = 'tmp/'

In [9]:
train_ids = np.load(DATA_PATH + 'tmp/train_lm.npy')
test_ids  = np.load(DATA_PATH + 'tmp/test_lm.npy')
itos = pickle.load(open(DATA_PATH + 'tmp/itos.pkl', 'rb'))

In [10]:
vs=len(itos)
vs,len(train_ids)

(1504, 1000)

## Wikitext103

We use a transfer learning method to learn the structure of the spanish language. In transfer learning (a fairly recent idea for NLP) a pre-trained LM that has been trained on a large generic corpus(like wikipedia articles) can be used to transfer it's knowledge to a target LM and the weights can be fine-tuned.

Our source LM is the wikitext103 LM created by Stephen Merity @ Salesforce research. Link to dataset The language model for wikitext103 (AWD LSTM) has been pre-trained and the weights can be downloaded here: Link. Our target LM is the IMDB LM.

In [11]:
from fastai.text import *

WIKITEX = '../database/ulmfit/'

In [15]:
#!wget -nH -r -np -P ../database/ulmfit/model/ http://files.fast.ai/models/wt103/fwd_wt103.h5

--2019-01-21 02:32:24--  http://files.fast.ai/models/wt103/fwd_wt103.h5
Resolviendo files.fast.ai (files.fast.ai)... 67.205.15.147
Conectando con files.fast.ai (files.fast.ai)[67.205.15.147]:80... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 462387687 (441M) [text/plain]
Guardando como: “../database/ulmfit/model/models/wt103/fwd_wt103.h5”

models/wt103/fwd_wt 100%[===================>] 440,97M  1,50MB/s    en 5m 40s  

2019-01-21 02:38:04 (1,30 MB/s) - “../database/ulmfit/model/models/wt103/fwd_wt103.h5” guardado [462387687/462387687]

ACABADO --2019-01-21 02:38:04--
Tiempo total de reloj: 5m 40s
Descargados: 1 ficheros, 441M en 5m 40s (1,30 MB/s)


In [18]:
#!wget -nH -r -np -P ../database/ulmfit/model/ http://files.fast.ai/models/wt103/itos_wt103.pkl

--2019-01-21 02:40:43--  http://files.fast.ai/models/wt103/itos_wt103.pkl
Resolviendo files.fast.ai (files.fast.ai)... 67.205.15.147
Conectando con files.fast.ai (files.fast.ai)[67.205.15.147]:80... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 4161252 (4,0M) [text/plain]
Guardando como: “../database/ulmfit/model/models/wt103/itos_wt103.pkl”

models/wt103/itos_w 100%[===================>]   3,97M  1,76MB/s    en 2,3s    

2019-01-21 02:40:46 (1,76 MB/s) - “../database/ulmfit/model/models/wt103/itos_wt103.pkl” guardado [4161252/4161252]

ACABADO --2019-01-21 02:40:46--
Tiempo total de reloj: 2,5s
Descargados: 1 ficheros, 4,0M en 2,3s (1,76 MB/s)


In [12]:
em_sz,nh,nl = 400,1150,3

In [13]:
PRE_PATH    = WIKITEX + 'models/wt103/'
PRE_LM_PATH = PRE_PATH + 'fwd_wt103.h5'

In [14]:
wgts = torch.load(str(PRE_LM_PATH), map_location=lambda storage, loc: storage)

In [15]:
enc_wgts = to_np(wgts['0.encoder.weight'])
row_m    = enc_wgts.mean(0)

In [16]:
itos2 = pickle.load(open(PRE_PATH + 'itos_wt103.pkl','rb'))
stoi2 = collections.defaultdict(lambda:-1, {v:k for k,v in enumerate(itos2)})

In [17]:
new_w = np.zeros((vs, em_sz), dtype=np.float32)
for i,w in enumerate(itos):
    r = stoi2[w]
    new_w[i] = enc_wgts[r] if r>=0 else row_m

In [18]:
wgts['0.encoder.weight'] = T(new_w)
wgts['0.encoder_with_dropout.embed.weight'] = T(np.copy(new_w))
wgts['1.decoder.weight'] = T(np.copy(new_w))

/home/jose/.local/lib/python3.6/site-packages/torch/cuda/__init__.py:97: UserWarning: 
    Found GPU0 GeForce GTX 750 Ti which is of cuda capability 5.0.
    PyTorch no longer supports this GPU because it is too old.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


## Training A new IMDB language model

In [19]:
wd=1e-7
bptt=70
bs=52
opt_fn = partial(optim.Adam, betas=(0.8, 0.99))

In [20]:
ULMFIT_PATH = DATA_PATH + 'ulmfit/'

In [21]:
trn_dl = LanguageModelLoader(np.concatenate(train_ids), bs, bptt)
val_dl = LanguageModelLoader(np.concatenate(test_ids), bs, bptt)
md     = LanguageModelData(ULMFIT_PATH, 1, vs, trn_dl, val_dl, bs=bs, bptt=bptt)

In [22]:
drops = np.array([0.25, 0.1, 0.2, 0.02, 0.15])*0.7

In [23]:
learner= md.get_model(opt_fn, em_sz, nh, nl, dropouti=drops[0], dropout=drops[1], wdrop=drops[2], dropoute=drops[3], dropouth=drops[4])

learner.metrics = [accuracy]
learner.freeze_to(-1)

In [24]:
learner.model.load_state_dict(wgts)

In [25]:
lr=1e-3
lrs = lr

In [27]:
learner.fit(lrs/2, 1, wds=wd, use_clr=(32,2), cycle_len=1)

  0%|          | 0/3 [00:00<?, ?it/s]


RuntimeError: cuda runtime error (8) : invalid device function at /pytorch/torch/lib/THC/generic/THCTensorMathPairwise.cu:102